Uppgift 03
Spamdetektering med engelska ord. Utökad tillämpning av textklassificering. Upphovspersoner = Colab - Chatgpt - Gemini - Axel

Här ligger datasetet, som du direkt laddar in i Colab utan omvägar: https://huggingface.co/datasets/ucirvine/sms_spam

Du skall kort fylla i vissa uppgifer i ett Worddokument. Och ladda upp det med lösningen.

Steg 1 - Installera komponenter som behövs för Tensorflow. För publicering på nätet. Får du en varning om att starta om sessionen. Gör det, och och kör om från början!
Fungerande Workaround för Tensorflow 2 (Webben)

In [ ]:
# @title Lösning: Flera textfiler + Grafer + Legacy Keras
import os

# 1. INSTALLERA TENSORFLOWJS
# AXELS ANMÄRKNING. Efter första körningen kan du dektivera installen genom att
# sätta ett # först på rad 7. Men GLÖM INTE DET SEN
!pip install tensorflowjs

# 2. TVINGA KERAS 2 (Legacy Mode - VIKTIGT FÖR ATT UNDVIKA FEL)
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ["TF_KERAS"] = "1"

import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
import pandas as pd
import json
import shutil
import zipfile
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files

print(f"TensorFlow version: {tf.__version__}")
print()

Ladda dataset, och förbered data. Vi hämtar datasetet direkt via ett anrop till ucirvine/sms_spam" se rad 9 i  koden. Inget behöver laddas upp i filer.
Uppgift 1 - Ta ett skärmklipp på resultatet av cellen

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print("TensorFlow:", tf.__version__)

ds = load_dataset("ucirvine/sms_spam")
texts = ds["train"]["sms"]

labels_raw = ds["train"]["label"]
def to01(y):
    if isinstance(y, (int, np.integer)):
        return int(y)
    y = str(y).lower().strip()
    return 1 if y == "spam" else 0

labels = np.array([to01(y) for y in labels_raw], dtype=np.int32)

X_train, X_test, y_train, y_test = train_test_split(
    np.array(texts, dtype=object),
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

print(f"Totalt antal poster: {len(texts)}")
print("Train:", X_train.shape, "Test:", X_test.shape)
print()
print("Spam-andel i train:", y_train.mean().round(3))

För att få en uppfattning om hur datan ser ut så listas de 10 första posterna i datasetet ut. Alltså av det som har hämtats.
Spam = Spammeddelande Ham = Inte spam

In [ ]:
print("De 10 första posterna i datasetet:")
for i in range(10):
    print(f"Text: {texts[i].strip()} - Label: {'Spam' if labels[i] == 1 else 'Ham'}")

Tokenizer + padding (sparas för webben).Då krävs dessa filer för att modellen skall fungera. Det du skriver in på webbsidan, omvandlas till siffror, som sedan körs i modellen och testa. Och du får resultatet. Se längst ned för ett exempel.

In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 8000        # lagom för gymnasieprojekt
MAX_LEN = 30             # SMS är korta
OOV_TOKEN = "<OOV>"

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN, lower=True)
tokenizer.fit_on_texts(X_train.tolist())

X_train_seq = tokenizer.texts_to_sequences(X_train.tolist())
X_test_seq  = tokenizer.texts_to_sequences(X_test.tolist())

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
X_test_pad  = pad_sequences(X_test_seq,  maxlen=MAX_LEN, padding="post", truncating="post") # Osedd testdata

oov_index = tokenizer.word_index.get(OOV_TOKEN, 1)

# Spara tokenizer + metadata till webben
with open("tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer.to_json())

# För enklare JS: spara word_index separat
with open("word_index.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer.word_index, f)

with open("model_info.json", "w", encoding="utf-8") as f:
    json.dump({
        "vocab_size": VOCAB_SIZE,
        "max_len": MAX_LEN,
        "oov_index": int(oov_index),
        "labels": ["ham", "spam"]
    }, f, indent=2)

print("Exempel (text):", X_train[0])
print("Exempel (sekvens):", X_train_seq[0][:10])
print("OOV index:", oov_index)
print()


Exempel (text): Ok i am on the way to home hi hi

Exempel (sekvens): [51, 2, 60, 19, 6, 132, 3, 78, 101, 101]
OOV index: 1


Uppgift för dig.
Nu har ordförrådet byggts upp i tokenizern. Nu kan skriva in engelsk mening. Ord som är vanligt förekommande får ett lågt indexvärde, och sådana som inte finns med ges värdet 1 = oov = out of words. Jämför den första uppgiften du gjorde. Ju fler oov, desto sämre resultat av modellen.

Uppgift 2 - Skriv en mening på engelska och ta ett skärmklipp av utfallet med tokens på den engelska respektive svenska meningen. Din slutsats?

Skriv in en engelsk mening nedan för att se hur den tokeniseras:

In [ ]:
english_sentence = input("Ange en engelsk mening: ")
english_sequence = tokenizer.texts_to_sequences([english_sentence])
print()
print(f"Din engelska mening: {english_sentence}")
print(f"Tokeniserad sekvens (engelska): {english_sequence[0]}")

Skriv nu in en svensk mening för att se hur den tokeniseras av samma tokenizer (observera att denna tokenizer är tränad på engelska data):

In [ ]:
swedish_sentence = input("Ange en svensk mening: ")
swedish_sequence = tokenizer.texts_to_sequences([swedish_sentence])
print()
print(f"Din svenska mening: {swedish_sentence}")
print(f"Tokeniserad sekvens (svenska): {swedish_sequence[0]}")
print("Var det med många 1.or = Ordet finns inte med")
print()
print("Modellen fungerar inte så bra (inte alls) på svenska ord och meningar...")

Nu är all data förbered så att den kan användas för träning i ett neural träningsmodell. Se separat info om den. Men lite kort! Se kommentarer i

In [ ]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32),
    keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=64),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

cb = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    # Här stoppas träningen med automatik. När den inte ger något mer. Inte alltid sant. Skydd mot överträning
    # Om du kollar på rad 27. Så ser du max antalet epoker.
    # Om du kollar på rad 29 så ser du batch-size. Antal data som matas in åt gången

]

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    callbacks=cb,
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=0)
print("Test accuracy:", round(float(test_acc), 4))


Visa träningen visuellet
Uppgift 3 - Ta ett skärmklipp på träningen. Din åsikt?

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.figure(figsize=(12, 6))

# Plot loss
plt.subplot(1, 2, 1) # 1 row, 2 columns, first plot
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot accuracy
plt.subplot(1, 2, 2) # 1 row, 2 columns, second plot
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Fundera på. Är träningen bra?

Visa i textbaserad Confusion Matrix.
Uppgift 4. Ta ett skärmklipp. Verkar modellen vara bra?

In [ ]:
y_prob = model.predict(X_test_pad, verbose=0).reshape(-1)
y_pred = (y_prob >= 0.5).astype(np.int32)

print(classification_report(y_test, y_pred, target_names=["ham","spam"]))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Ett confusion chart i diagram

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

Förbered modellen för publicering på webben

In [ ]:
!rm -rf tfjs_model
model.save("spam_model.keras")

!tensorflowjs_converter \
  --input_format=keras \
  --output_format=tfjs_layers_model \
  spam_model.keras tfjs_model

!ls -la tfjs_model


In [ ]:
# Spara filerna i rätt format
output_dir = 'tfjs_model_multi_files'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

tfjs.converters.save_keras_model(model, output_dir)

# Define variables needed for metadata
word_index = tokenizer.word_index
max_length = MAX_LEN
padding_type = "post"
trunc_type = "post"
label_map = ["ham", "spam"] # Based on the definition in model_info.json
oov_tok = OOV_TOKEN

metadata = {
    'word_index': word_index,
    'max_length': max_length,
    'padding_type': padding_type,
    'trunc_type': trunc_type,
    'labels': label_map,
    'oov_token': oov_tok
}
with open(os.path.join(output_dir, 'metadata.json'), 'w', encoding='utf-8') as f:
    json.dump(metadata, f)

zip_filename = 'tfjs_multi_files.zip'
!zip -r {zip_filename} {output_dir}

print(f"\n✅ KLAR. Laddar ner '{zip_filename}'.")
files.download(zip_filename)
print()

Verifiering av filerna. Kommer de att fungera på Github med hjälp av
TFJS: 4.22.0. Detta är ett vanligt problem med webbpublicering. Se svaret längst ned!
Uppgift 5. Ta ett skärmklipp av resultatet

In [ ]:
import os
import zipfile
import json
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs # Required for loading the converted model
import shutil # For cleaning up temporary directory

# 1. Define paths and unzip the files
zip_filename = 'tfjs_multi_files.zip'
output_dir_verification = 'tfjs_temp_for_verification'

# Clean up previous verification directory if it exists
if os.path.exists(output_dir_verification):
    shutil.rmtree(output_dir_verification)
os.makedirs(output_dir_verification, exist_ok=True)

print(f"Unzipping '{zip_filename}' to '{output_dir_verification}'...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(output_dir_verification)

# The actual TFJS model files are inside 'tfjs_model_multi_files' within the zip
tfjs_model_sub_path = os.path.join(output_dir_verification, 'tfjs_model_multi_files')
metadata_path = os.path.join(tfjs_model_sub_path, 'metadata.json')
model_json_path = os.path.join(tfjs_model_sub_path, 'model.json')

print(f"Successfully unzipped files to: {tfjs_model_sub_path}")
print(f"Contents of '{tfjs_model_sub_path}': {os.listdir(tfjs_model_sub_path)}")

# 2. Load metadata
print("\n--- Loading Metadata (tokenizer information) ---")
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

word_index = metadata['word_index']
max_length = metadata['max_length']
oov_token = metadata['oov_token']
labels = metadata['labels']
oov_index = word_index.get(oov_token, 1) # Get the actual numerical index for OOV

print(f"Max Length: {max_length}")
print(f"OOV Token: '{oov_token}' (Index: {oov_index})")
print(f"Labels: {labels}")

# 3. Load the TFJS converted model back into Keras for verification
print("\n--- Attempting to load TFJS model (model.json and bin files) into Keras ---")
try:
    loaded_tfjs_model = tfjs.converters.load_keras_model(model_json_path)
    print("✅ Successfully loaded TFJS model into Keras!")
    loaded_tfjs_model.summary()
except Exception as e:
    print(f"❌ Error loading TFJS model: {e}")
    # If the model cannot be loaded, the verification process stops here.
    # Clean up and exit
    shutil.rmtree(output_dir_verification)
    raise

# 4. Create a preprocessing function mimicking client-side (JavaScript) logic
def preprocess_text_for_web(text, word_index, max_length, oov_index):
    # Tokenize: Split text into words, convert to lowercase
    words = text.lower().split()
    sequence = []
    for word in words:
        # Get index, use OOV index if not found
        sequence.append(word_index.get(word, oov_index))

    # Pad/Truncate sequence
    if len(sequence) > max_length:
        processed_sequence = sequence[:max_length] # Truncate from the end
    else:
        processed_sequence = sequence + [0] * (max_length - len(sequence)) # Pad with zeros

    # Ensure it's a numpy array of integers for the model
    return np.array([processed_sequence], dtype=np.int32)

# 5. Make predictions using the loaded model and preprocessor
def make_prediction_with_web_model(text, model, word_index, max_length, oov_index, labels):
    processed_input = preprocess_text_for_web(text, word_index, max_length, oov_index)
    prediction_prob = model.predict(processed_input, verbose=0)[0][0]
    prediction_label_index = (prediction_prob >= 0.5).astype(int)
    predicted_label = labels[prediction_label_index]
    return prediction_prob, predicted_label

print("\n---  Testar förutsägeler med modellen ---")
test_sentences = [
    "Congratulations! You have won a free iPhone. Claim your prize now!",  # Spam example
    "Hi, how are you doing today? I wanted to check in about our meeting.",               # Ham example
    "URGENT! Your account needs verification. Click here to avoid suspension.", # Another spam
    "Could you please send me the report by end of day? Thanks!"             # Another ham
]

for sentence in test_sentences:
    prob, label = make_prediction_with_web_model(sentence, loaded_tfjs_model, word_index, max_length, oov_index, labels)
    print(f"Text: '{sentence}'")
    print(f"  Prediction: {label} (Probability: {prob:.4f})")

# 6. Clean up temporary files
print("\n--- Cleaning up temporary files ---")
shutil.rmtree(output_dir_verification)
print(f"Cleaned up temporary directory: {output_dir_verification}")
print()
print("\n✅ Verification complete. Testerna säger att de skapade filerna skall fungera. Vi får väl se")
print()
print("Ladda ned zipfilen från files /content/tfjs_multi_files.zip")